In [175]:
import numpy as np

In [176]:
def sigmoid(x):
    return np.round((1 / (1 + np.exp(-x))), 8)


def d_sigmoid(x):
    return np.round((np.exp(-x) / np.power((1 + np.exp(-x)), 2)), 8)


def generate_weight(source_units, target_units):
    return np.array(
        [np.round(np.random.randn(), 8) for _ in range(source_units * target_units)]
    ).reshape(target_units, source_units)

In [177]:
def forward_propagation(prototype, weight):
    """Calcule les valeurs d'entrée et de sortie des unités de chaque couche
    Args:
        prototype (list[float]): prototype qui entre dans les noeuds de la couche d'entrée
        weight (list[list[float]]): poids de chaque connexions entre les noeuds des couches
            eg: Pour un réseau de type [3, 2, 1] w doit ressembler à
                w = [[[w11, w12, w13], [w21, w22, w23]], [[w11, w12]]]
    Returns:
        units_input (list[list[float]]): chaque valeurs d'entrée des unités de chaque couche
            eg: Pour un réseau de type [3, 2, 1] on a [[h21, h22], [h31]]
        units_output (list[list[float]]): chaque valeurs de sortie des unités de chaque couche
            eg: Pour un réseau de type [3, 2, 1] on a [[V11, V12, V13], [V21, V22], [V31]]
    """
    units_input = []
    units_output = [prototype]
    for layer in weight:
        current_input = []
        current_output = []
        for unit in layer:
            h = np.round(np.dot(prototype, unit), 8)
            current_input.append(h)
            current_output.append(sigmoid(h))
        units_input.append(current_input)
        units_output.append(current_output)
        prototype = current_output
    return (units_input, units_output)

In [178]:
def output_layer_delta(input, output, desired_output):
    """Calcule les deltas pour la couche de sortie
    Args:
        input (list[float]): valeurs d'entrées des unités de sortie
        output (list[float]): valeurs produites par les unités de sortie
        desired_output (list[float]): valeurs désirés
    Returns:
        units_delta (list[float]): deltas des unités de la couche de sortie
    """
    units_delta = []
    for i in range(len(desired_output)):
        delta = d_sigmoid(input[i]) * (desired_output[i] - output[i])
        units_delta.append(np.round(delta, 8))
    return units_delta

In [179]:
def middle_layer_delta(input, weight, succ_delta):
    """Calcule les deltas pour les couches précédentes
    Args:
        input (list[list[float]]): chaque valeurs d'entrée des unités de chaque couche
        weight (list[list[float]]): poids de chaque connexions entre les noeuds des couches
        succ_delta (list[list[float]]): delta des couches suivants
    Returns:
        succ_delta (list[list[float]]): liste de tous les deltas
    """
    units_delta = []
    for i in range(len(weight) + 1, 2, -1):
        for j in range(len(weight[i - 3])):
            delta = d_sigmoid(input[i - 3][j]) * np.dot(
                [weight[i - 2][k][j] for k in range(len(weight[i - 2]))], succ_delta[-1]
            )
            units_delta.append(np.round(delta, 8))
        succ_delta.append(units_delta)
    return succ_delta

In [180]:
def delta_weight(learning_step, succ_layer_delta, output):
    """Calcule  delta weight pour mettre à jour les poids
    Args:
        learning_step (float): pas d'apprentissage
        succ_layer (list[list[float]]): tous les deltas de chaque noeud de chaque couche
        output (list[list[float]]): valeus produites par tous les noeuds
    Returns:
        delta_weights (list[list[float]]): delta de tous les noeuds de chaque couche
    """
    output_copy = list.copy(output)
    output_copy.pop()
    output_copy.reverse()
    delta_weights = []
    for i in range(len(succ_layer_delta)):
        current_delta = []
        for delta in succ_layer_delta[i]:
            current_delta.append(
                np.round((np.multiply(delta * learning_step, output_copy[i])), 8)
            )
        delta_weights.append(current_delta)
    delta_weights.reverse()
    return delta_weights

In [181]:
def update_weight(weight, delta_weight):
    """Met à jours les poids des connexions
    Args:
        weight (list[list[float]]): poids des connexions
        delta_weight (list[list[float]]): delta de tous les noeuds de chaque couche
    Returns: poids des connexions à jour
    """
    return [np.round(np.add(weight[i], delta_weight[i]), 8) for i in range(len(weight))]

In [182]:
prototype = [1, 0, 1]
w = [[[0.2, 0.1, 0.1], [0.3, 0.2, 0.3]], [[0.2, 0.3]]]
learning_step = 0.1
desired_output = [1]

In [183]:
unit_inputs, unit_outputs = forward_propagation(prototype, w)
d = output_layer_delta(unit_inputs[-1], unit_outputs[-1], desired_output)
d = middle_layer_delta(unit_inputs, w, [d])
dw = delta_weight(learning_step, d, unit_outputs)
w = update_weight(w, dw)
w

[array([[0.20050546, 0.1       , 0.10050546],
        [0.30070958, 0.2       , 0.30070958]]),
 array([[0.20593883, 0.30667507]])]